### Proyecto 2A - Diseño de SCD2 y carga de la tabla

In [ ]:
Diseño del Data Flow
====================

        SOURCE                   DERIVED COLUMN                  EXISTS                    SURROGATE KEY                    JOIN                      DERIVED COLUMN                      SELECT                     ALTER ROW                       UNION                          SINK
 ___________________          ___________________          ___________________          ___________________          ___________________          _______________________          ___________________          ___________________          ______________________          ___________________              
|                   |        |                   |        |                   |        |                   |        |                   |        |                       |        |                   |        |                   |        |                      |        |                   |            
|      NEWData      |--------| CallNewHashValues |--------|  CheckForNewData  |--------|  AddSurrogateKey  |--------|    joinMaxSkey    |--------| CalculateExtraColumns |--------|  ReleventColumns  |--------|     InsertSet     |--------| UnionInsertAndUpdate |--------|    sinkToSQLDB    |
|___________________| +      |___________________| +      |___________________| +      |___________________| +      |___________________| +      |_______________________| +      |___________________| +      |___________________| +      |______________________| +      |___________________| 
                                                                                                                                                                                                                                               |             
        SOURCE              NEW BRANCH / DERIVED COLUMN               EXISTS                      DERIVED COLUMN                        SELECT                           ALTER ROW                                                             | 
 ___________________          ________________________          ____________________          _____________________          _____________________________          ___________________                                                        | 
|                   |        |                        |        |                    |        |                     |        |                             |        |                   |-------------------------------------------------------'               
|    ExistingData   |--------| CallExistingHashValues |--------| UpdateExistingData |--------| ChangeUpdatedStatus |--------| SelectReleventColumn_Update |--------|     UpdateSet     |
|___________________| +      |________________________| +      |____________________| +      |_____________________| +      |_____________________________| +      |___________________|

                                   DERIVED COLUMN                  SURROGATE KEY                                                               
                              ________________________          ___________________                                   
                             |                        |        |                   |                   
                             | CallExistingHashValues |--------|      MaxSkey      |
                             |________________________| +      |___________________| 




#### Paso 1 - Creación de la tabla target donde cargaremos nuestro data

1. Comenzamos creando la tabla **tbl_FIFAData** donde cargaremos nuestra data

<center><img src="https://i.postimg.cc/90v0J91B/adf351.png"></center>

#### Paso 2 - Creación de Linked Services

2.  En el Proyecto 1 se creó el Linked Services **LS_AzureDataLakeStorageG2** que utilizaremos para hacer referencia a nuestro ADLS
    
3.  Creamos un nuevo Linked Services llamado **LS_AzureSqlDatabase_IPLnew** que hará referencia a Azure SQL Database. Se utilizaron
    las mismas configuraciones que el Linked Services **LS_AzureSqlDatabase_IPL_db** utilizado en el Proyecto 1
    *   Servidor: db-ipl-demo
    *   Base de datos: db_IPL

#### Paso 3 - Creación de datasets

4.  ADLS tenemos un contenedor llamado **fifa** el cual contiene un archivo llamado **Fifa_1.csv**

<center><img src="https://i.postimg.cc/1td77RFH/adf352.png"></center>

5. Crearemos un dataset de origen utilizando el archivo **Fifa_1.csv**:
    *   Se llamará **adf_ds_Fifa_source_csv**
    *   Hará referencia a Azure Data Lake Storage Gen2
    *   Hará referencia a un tipo de archivo de texto delimitado (DelimitedText)

<center><img src="https://i.postimg.cc/W1X4DxgV/adf353.png"></center>
<center><img src="https://i.postimg.cc/kGvm0tyC/adf354.png"></center>

6. Crearemos un dataset de destino utilizando la tabla **tbl_FIFAData**:
    *   Se llamará **adf_ds_Fifa_Sql_table**
    *   Hará referencia a Azure SQL Database

<center><img src="https://i.postimg.cc/kXw60Tw2/adf355.png"></center>
<center><img src="https://i.postimg.cc/FH7R2N76/adf356.png"></center>

#### Paso 4 - Creación de Sources en el Data Flow

7.  Creamos la primera tarea de origen llamada **SourceFifaCSV** que toma como referencia el 
    dataset **adf_ds_Fifa_source_csv**

<center><img src="https://i.postimg.cc/6Q3zFHZc/adf357.png"></center>
<center><img src="https://i.postimg.cc/yNcW6K08/adf359.png"></center>
<center><img src="https://i.postimg.cc/2ypwbrK9/adf358.png"></center>

8.  Creamos la segunda tarea de origen llamada **ExistingData** que toma como referencia el 
    dataset **adf_ds_Fifa_Sql_table**

<center><img src="https://i.postimg.cc/W4kS2d9q/adf360.png"></center>
<center><img src="https://i.postimg.cc/J01QcPRj/adf361.png"></center>

#### Paso 5 - Creación de Parámetros a nivel de Data Flow

9. Creamos dos parámetros a nivel de Data Flow:
    *   **parameter_ID**: 'ID'                   (Tipo: String)
    *   **parameter_SCD**: 'Club,Wage,Position'  (Tipo: String)

<center><img src="https://i.postimg.cc/1R2TwVTX/adf362.png"></center>

#### Paso 6 - Agregar tarea "Derived Column" a ambas tareas de origen

9. Agregamos la tarea **Derived Column** al primer Source con el fin de crear dos columnas:
    *   **ID_Hash** --> md5(byName($parameter_ID)) --> md5(byName('ID'))
    *   **SCD_Hash** --> md5(byNames(split($parameter_ID,','))) -->  md5(byNames(['Club','Wage','Position']))

<center><img src="https://i.postimg.cc/02PvTtRg/adf369.png"></center>

10. Agregamos la tarea **Derived Column** al segundo Source con el fin de crear dos columnas:
    *   **ID_Hash** --> md5(byName($parameter_ID)) --> md5(byName('ID'))
    *   **SCD_Hash** --> md5(byNames(split($parameter_ID,','))) -->  md5(byNames(['Club','Wage','Position']))

<center><img src="https://i.postimg.cc/HLh5KcVK/adf370.png"></center>

11. Si visualizamos los datos previamente para el primer flujo:

<center><img src="https://i.postimg.cc/3w4WkLxS/adf371.png"></center>
<center><img src="https://i.postimg.cc/T193zFtw/adf372.png"></center>

#### Paso 7 - Utilizar "Exist" para validar datos de origen y destino

<center><img src="https://i.postimg.cc/rmG4phh6/adf373.png"></center>

#### Paso 8 - Utilizar "Surrogate Key" y calcular el maximo 'Surrogate Key'

<center><img src="https://i.postimg.cc/yYTzT0GV/adf374.png"></center>

12. Vamos a calcular el maximo 'Surrogate Key'
```
    MaxSkey: max(toInteger(byName('Skey')))
```
<center><img src="https://i.postimg.cc/CK5JznXM/adf375.png"></center>
<center><img src="https://i.postimg.cc/g2n6hyz7/adf376.png"></center>

#### Paso 9 - Joinear 'Max Surrogate Key' con un dataset nuevo o actualizado

13. La condición para realizar el CROSS JOIN
```
    MaxSkey==Skey||true()
```
<center><img src="https://i.postimg.cc/fyFn3hTk/adf377.png"></center>
<center><img src="https://i.postimg.cc/zGZ647Lm/adf378.png"></center>

#### Paso 10 - Agregamos una nueva tarea "Derived Column"

13. Modificamos la columna **Skey** y creamos 3 nuevas columnas
```
    Column              Expression              
    ------              ----------
    Skey:               Skey + iifNull(MaxSkey,0)
    Active              1
    ActiveStartTime     currentUTC()
    ActiveEndTime       toTimestamp(toString(null()))
```
<center><img src="https://i.postimg.cc/Dzj8C8xN/adf379.png"></center>
<center><img src="https://i.postimg.cc/x8QzbqgF/adf380.png"></center>

#### Paso 11 - Agregamos una tarea "Select" para seleccionar las columnas que utilizaremos

<center><img src="https://i.postimg.cc/4NDQJNRq/adf381.png"></center>
<center><img src="https://i.postimg.cc/ZR5rfW6B/adf382.png"></center>
<center><img src="https://i.postimg.cc/tR1rycV1/adf385.png"></center>
<center><img src="https://i.postimg.cc/SxLCk6rY/adf383.png"></center>
<center><img src="https://i.postimg.cc/TPRmwK77/adf384.png"></center>


#### Paso 12

<center><img src="https://i.postimg.cc/59TFd7w2/adf386.png"></center>
<center><img src="https://i.postimg.cc/BvmXhMSs/adf387.png"></center>
<center><img src="https://i.postimg.cc/mkHkg4Q2/adf388.png"></center>

#### Paso 13

<center><img src="https://i.postimg.cc/RFCDc1dj/adf389.png"></center>
<center><img src="https://i.postimg.cc/BbX7zqdp/adf390.png"></center>
<center><img src="https://i.postimg.cc/3J0bCxjJ/adf391.png"></center>
<center><img src="https://i.postimg.cc/Gmk7VgSW/adf392.png"></center>

#### Paso 14

<center><img src="https://i.postimg.cc/6qfSRjxJ/adf393.png"></center>
<center><img src="https://i.postimg.cc/bwB5QSWJ/adf394.png"></center>

#### Paso 15

<center><img src="https://i.postimg.cc/FKFjsTDk/adf395.png"></center>
<center><img src="https://i.postimg.cc/cC335242/adf396.png"></center>
<center><img src="https://i.postimg.cc/pXNznWsH/adf397.png"></center>

#### Paso 16 - Agregar un "sink" y ejecutar nuestro Pipeline

<center><img src="https://i.postimg.cc/7hCvv1pp/adf398.png"></center>
<center><img src="https://i.postimg.cc/NGVq6VXY/adf399.png"></center>

Validamos y ejecutamos nuestro pipeline utilizando **Debug**

<center><img src="https://i.postimg.cc/W33Cr6bh/adf402.png"></center>

Visualizemos los registros del archivo **Fifa_1.csv**

<center><img src="https://i.postimg.cc/mgfvqHV5/adf401.png"></center>

Podemos ver que todos los registros del archivo **Fifa_1.csv** se han cargado en la tabla, dado que
la tabla estaba vacía

<center><img src="https://i.postimg.cc/gkZ9NDyh/adf400.png"></center>

#### Paso 17 - Pruebas unitarias: Validar la ejecución del pipeline Paso 1

Tenemos un nuevo archivo **File_2.csv** el cual posee un solo registro y es similar a la fila con SKey = 2 e ID = PL4 que
se encuentra ya cargada en la tabla. Sin embargo, el valor del campo **Name** es distinto

<center><img src="https://i.postimg.cc/g2w2m8Pq/adf403.png"></center>

Modificamos el dataset de origen **ads_ds_Fifa_source.csv** y cambiamos la referencia hacia al archivo **File_2.csv**

<center><img src="https://i.postimg.cc/4d1JB1zZ/adf404.png"></center>

Luego de Validar y ejecutar nuestro pipeline utilizando Debug, podemos ver que no hubo ningún cambio en la tabla

<center><img src="https://i.postimg.cc/J7PrXPr5/adf405.png"></center>

#### Paso 18 - Pruebas unitarias: Validar la ejecución del pipeline Paso 2

Tenemos un nuevo archivo **File_3.csv** el cual posee un solo registro y es similar a la fila con SKey = 3 e ID = PL1 que
se encuentra ya cargada en la tabla. Sin embargo, el valor del campo **Club** y **Wage** es distinto

<center><img src="https://i.postimg.cc/Wb85Tq73/adf406.png"></center>

Modificamos el dataset de origen **ads_ds_Fifa_source.csv** y cambiamos la referencia hacia al archivo **File_3.csv**

<center><img src="https://i.postimg.cc/tgK5239r/adf407.png"></center>

Luego de Validar y ejecutar nuestro pipeline utilizando Debug, podemos ver que se agregó el registro a la tabla. El valor de
la columna **SKey** comenzó nuevamente de 1.

<center><img src="https://i.postimg.cc/HxbtKrxT/adf408.png"></center>

#### Paso 19 - Pruebas unitarias: Validar la ejecución del pipeline Paso 3

Tenemos un nuevo archivo **File_4.csv** el cual posee dos registros y son similares a filas que ya se encuentran 
cargadas en la tabla. Sin embargo, para el primer registro los valores de los campos **Age** y **Wage** es distinto.
Para el segundo registro el valor del campo **Position** es distinto

<center><img src="https://i.postimg.cc/NGmgJBkb/adf409.png"></center>

Modificamos el dataset de origen **ads_ds_Fifa_source.csv** y cambiamos la referencia hacia al archivo **File_4.csv**

<center><img src="https://i.postimg.cc/yYT7Kj5Z/adf410.png"></center>

Luego de Validar y ejecutar nuestro pipeline utilizando Debug, podemos ver que se agregaron los registros a la tabla. El valor de
la columna **SKey** comenzó nuevamente de 1.

<center><img src="https://i.postimg.cc/DwjFJZ7f/adf411.png"></center>